In [ ]:
#query for efi2 deficit for Faecal incontinence, using updated code for efalls deficits
 CREATE OR REPLACE TABLE `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_efalls_fi` as (
#bring all snoemd and ctv3codes for care home cohort in primary into the cte
with a as (
  select
  person_id,
  ctv3code,
  snomedcode,
  dateevent
  from `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  where person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)),
  #join care home entry date from msater table
  a_master as (
    select
    a.*,
    cast(j.first_episodestartdate as datetime) first_episodestartdate
    from a
    left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` j on
    a.person_id = j.person_id),
#filter for Faecal incontinence snomed codes
  b as (
    select
    person_id,
    case
    when snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Faecal incontinence') AND dateevent < first_episodestartdate then 'true'
    when ctv3code in(select CTV3 from `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Faecal incontinence') AND dateevent < first_episodestartdate then 'true' end as faecal_incontinence,
    from a_master)

#apply dummy variable
    select
     distinct person_id,
    faecal_incontinence
    from b
    where faecal_incontinence = 'true'
    order by person_id)
  



